# Panimula sa Prompt Engineering

Ang prompt engineering ay ang proseso ng pagdidisenyo at pag-optimize ng mga prompt para sa mga gawain sa natural language processing. Kabilang dito ang pagpili ng tamang mga prompt, pag-tune ng kanilang mga parameter, at pagsusuri ng kanilang performance. Mahalaga ang prompt engineering para makamit ang mataas na accuracy at efficiency sa mga NLP model. Sa seksyong ito, tatalakayin natin ang mga pangunahing kaalaman sa prompt engineering gamit ang mga modelo ng OpenAI para sa eksplorasyon.


### Ehersisyo 1: Tokenization
Galugarin ang Tokenization gamit ang tiktoken, isang open-source na mabilis na tokenizer mula sa OpenAI
Tingnan ang [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) para sa iba pang mga halimbawa.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Ehersisyo 2: Suriin ang Setup ng Github Models Key

Patakbuhin ang code sa ibaba para tiyakin na tama ang pagkaka-setup ng iyong Github Models endpoint. Sinusubukan lang ng code ang isang simpleng prompt at tinitingnan kung tama ang sagot. Ang input na `oh say can you see` ay dapat kumpletuhin ng sagot na katulad ng `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Ehersisyo 3: Mga Gawa-gawang Sagot

Suriin kung ano ang mangyayari kapag hiningi mo sa LLM na magbigay ng sagot para sa isang prompt tungkol sa isang paksa na maaaring hindi umiiral, o tungkol sa mga paksang maaaring hindi nito alam dahil wala ito sa pre-trained dataset nito (mas bago). Tingnan kung paano nagbabago ang sagot kung susubukan mo ang ibang prompt, o ibang modelo.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Ehersisyo 4: Batay sa Instruksyon
Gamitin ang "text" na variable para itakda ang pangunahing nilalaman
at ang "prompt" na variable para magbigay ng instruksyon na may kaugnayan sa pangunahing nilalaman.

Dito, hinihiling natin sa modelo na ibuod ang teksto para sa isang estudyante sa ikalawang baitang


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Ehersisyo 5: Kumplikadong Prompt
Subukan ang isang request na may system, user, at assistant na mga mensahe
Ang System ang nagse-set ng context ng assistant
Ang mga mensahe ng User at Assistant ay nagbibigay ng konteksto para sa multi-turn na usapan

Pansinin kung paano na-set ang personalidad ng assistant bilang "sarcastic" sa system context.
Subukan gumamit ng ibang personalidad na context. O kaya subukan ang ibang serye ng input/output na mga mensahe


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Ehersisyo: Tuklasin ang Iyong Intuwisyon
Ang mga halimbawa sa itaas ay nagbibigay sa iyo ng mga pattern na maaari mong gamitin para gumawa ng mga bagong prompt (simple, kumplikado, may tagubilin, atbp.) – subukan mong gumawa ng iba pang mga ehersisyo para tuklasin ang iba pang mga ideya na napag-usapan natin tulad ng mga halimbawa, pahiwatig, at iba pa.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagaman nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring lumitaw mula sa paggamit ng pagsasaling ito.
